In [1]:
import sys
sys.path.append('..')

from ml.dataset.corpus import *
from ml.dataset.mappers_preprocess import *
from ml.dataset.mappers_simplify import *
from ml.dataset.mappers_prepare import *
from ml.dataset.processor import *
from ml.dataset.pipeline import *
from ml.dataset.model import *
from IPython.core.display import HTML, display

import matplotlib.pyplot as plt
import pandas as pd
import pychord
import music21
music21.environment.set('musescoreDirectPNGPath', '/usr/bin/musescore')

In [2]:
X = np.load('../datasets/simple/X.npy')
y = np.load('../datasets/simple/y.npy')

In [3]:
pred = Predictor()

In [4]:
pred.X = X
pred.y = y
pred.load_dicts('../datasets/simple/simple_dataset_dicts.pickle')

In [5]:
pred.encode()

Shape:  (44092, 16) (44092, 8)
Classes: ['' '-' 'A' 'B-' 'C' 'D' 'E' 'E-' 'F' 'G']


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(pred.X, pred.y)

In [7]:
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import accuracy_score
from nottingham_test import custom_scorers
from sklearn.metrics import make_scorer

pred.fit( RandomForestClassifier(n_estimators=20), X_train, y_train)

In [8]:
y_pred = pred.predict(X_test)

In [9]:
print(accuracy_over_axis(y_test, y_pred))

0.7777374580422752


In [10]:
pred.dump_clf('../datasets/simple/rf.pickle')

In [50]:
corpus = SongCorpus()
corpus.load_from_file('../datasets/simple/simple_dataset_SimplifyChordsMapper.pickle',max_count=100)

In [51]:
song = corpus.songs[40]

In [ ]:
song.get_music21_repr().show()

In [54]:
song.get_music21_repr().show('midi')

In [ ]:
predicted_song = pred.predict_song(song)

In [ ]:
predicted_song.get_music21_repr().show()

In [57]:
predicted_song.get_music21_repr().show('midi')